In [ ]:
#!import BenchmarkAnalysis.dib

In [ ]:
// This only needs to be evaluated when iterating on the Reports-Impl code.

#!import Reports-Impl.dib

# Examples

Note that this is just the state of my working notebook, not a comprehensive set of examples.

In [ ]:
string rootDir = @"C:\home\repro\decommit";
var hugeDM = new DataManager();
foreach (string config in ML("rcbase", "rc1", "alt-draft-log3"))
{
    string configDir = $"gcperfsim_{config}_gc";
    foreach (string benchBase in ML("100mb", "100mb-10mb", "10mb-100mb"))
    {
        foreach (string benchSuffix in ML("", "-low"))
        {
            string bench = benchBase + benchSuffix;
            string etl = $@"{rootDir}\{configDir}\{bench}\{bench}.{config}.0.etl";
            if (!File.Exists(etl))
            {
                Console.WriteLine($"{etl} does not exist");
                continue;
            }
            hugeDM.AddGCTrace(etl, ML("corerun"), config: config, loadMultipleProcesses: false);
        }
    }
}

In [ ]:
hugeDM

In [ ]:
string sds_process_name = "Microsoft.M365.Core.Sds.Service";
string store_worker_process_name = "Microsoft.Exchange.Store.Worker";
var store_work_process_list = ML(sds_process_name, store_worker_process_name);
var sdsDM = DataManager.CreateGCTraces(@"c:\home\repro\notebook-gcapi", store_work_process_list);

(sdsDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.ProcessID,
sdsDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.CommandLine)

In [ ]:
var diffDataManager = DataManager.CreateAspNetData(ML(
    @"C:\home\repro\hc\asp_traceplus3_gc",
    @"C:\home\repro\hc\asp_tp3-m4_gc",
    @"C:\home\repro\hc\asp_slope_gc",
    @"C:\home\repro\hc\asp_evaldecr_gc"
));

In [ ]:
var cardsDM = DataManager.CreateGCTraces(@"c:\home\repro\2401310010004275", pertinentProcesses: ML("EXCEL"));

In [ ]:
var low4DM = DataManager.CreateAspNetData(@"c:\home\repro\hc\asp_v2-fixrearranged-mult-max_gc"
    //, benchmarkFilter: Filter.RE("Stage.*|Json.*");
    //, iterationFilter: IntFilter.Values(1)
);

(low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs.Keys,
 low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs["v2-fixrearranged-mult-max-h4"].Benchmarks["Stage2"].Iterations[1].GCSummaryInfo.MaxHeapCount,
 low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs["v2-fixrearranged-mult-max"].Benchmarks["Stage2"].Iterations[1].GCSummaryInfo.MaxHeapCount,
 low4DM.GetConfigs(Filter.All, Filter.RE("max")))


In [ ]:
var rc3DataManager = DataManager.CreateAspNetData(@"C:\home\repro\hc\asp_v2-fixrearranged_gc");
rc3DataManager.AddAspNetData(@"C:\home\repro\hc\asp_v2-tune_gc");

rc3DataManager.GetConfigs(Filter.All, Filter.All)

In [ ]:
var net6dm = DataManager.CreateGCTrace(@"C:\home\repro\2046032\Microsoft.MWC.Workload.OneLake.Service.EntryPoint.exe_WithGCon.nettrace", ML("Microsoft.MWC.Workload.OneLake.Service.EntryPoint"));
net6dm.AddGCTrace(@"C:\home\repro\2046032\TESTenvCST350 with perf.nettrace", ML("Microsoft.MWC.Workload.OneLake.Service.EntryPoint"));

Charting examples

In [ ]:
ChartBenchmarks(sdsDM, ML(Metrics.B.AveragePercentPauseTimeInGC, Metrics.B.MaxHeapCount)
    //, benchmarkFilter: Filter.Names("JsonMapAction")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    );

In [ ]:
TableBenchmarks(low4DM, ML(Metrics.B.AveragePercentPauseTimeInGC, Metrics.B.MaxHeapCount)
    //, benchmarkFilter: Filter.Names("JsonMapAction")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    );

In [ ]:
hugeDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.GCs

In [ ]:
ChartGCData(hugeDM, ML(Metrics.G.CommittedBeforeInUse, Metrics.G.CommittedBeforeInFree, Metrics.G.CommittedBeforeInGlobalFree, Metrics.G.CommittedBeforeInGlobalDecommit)
  , configNameSimplifier: NameSimplifier.PrefixDashed
  , debug: true
  );

In [ ]:
foreach (var xarr in new[] { (XArrangement) XArrangements.Default, XArrangements.Sorted, XArrangements.CombinedSorted })
{
TableBenchmarks(low4DM,
    ML(Metrics.B.MaxHeapCount,
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Min),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountDirectionChanges, Aggregation.Range)),
    configNameSimplifier: NameSimplifier.PrefixDashed,
    xArrangement: xarr,
    configFilter: new Filter(excludeRE: "h4")
    //, debug: true
    );
}

In [ ]:
foreach (var xarr in new[] { (XArrangement) XArrangements.Default, XArrangements.Sorted, XArrangements.CombinedSorted })
{
TableBenchmarks(diffDataManager,
    ML((Metrics.B.MaxHeapCount),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Min),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountDirectionChanges, Aggregation.Range)),
    //configNameSimplifier: NameSimplifier.PrefixDashed,
    xArrangement: xarr);
}

In [ ]:
TableIterations(diffDataManager, Metrics.I.MaxHeapCount
    , configFilter: Filter.Names("traceplus3", "tp3-m4")
);

In [ ]:
foreach (var lat in Metrics.I.LatencyMSList)
{
    TableBenchmarks(diffDataManager, Metrics.Promote(lat, Aggregation.Average)
        // , benchmarkFilter: Filter.RE("Stage.*")
        // configNameSimplifier: NameSimplifier.PrefixDashed,
        // types: B_XType.All,
        //configFilter: new Filter(excludeRE: ".*h4")
        );
}

In [ ]:
ChartIterations(diffDataManager, ML(/*Metrics.I.GCScore,*/ Metrics.I.RequestsPerMSec));

In [ ]:
TableGCData(net6dm,
    ML(
        Metrics.G.HeapSizeAfter,
        Metrics.G.HeapSizeBefore,
        new Metric<TraceGC>(gc => gc.HeapStats.GCHandleCount, "GC Handles", "#"),
        Metrics.G.GenLargeSizeBefore,
        Metrics.G.GenLargeSizeAfter,
        Metrics.G.GenLargeObjSizeAfter,
        new Metric<TraceGC>(gc => gc.HeapStats.TotalHeapSize / 1000000, "Total heap size", "MB")
    )
    , textPresenter: TextPresenter.Html
    , configNameSimplifier: new ListSimplifier(("Microsoft.MWC.Workload.OneLake.Service.EntryPoint.exe_WithGCon", "Entry"), ("TESTenvCST350 with perf", "TES"))
    , dataFilter: gc => gc.Number > 300 && gc.Number < 310
    );

In [ ]:
ChartGCData(low4DM
    , metrics: ML(Metrics.G.PauseDuration)
    , benchmarkFilter: Filter.Names("Stage2")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    , xMetric: Metrics.X.StartRelativeMSec
    , dataFilter: gc => gc.StartRelativeMSec > 2000
    , iterationFilter: IntFilter.Values(0)
    , xArrangement: XArrangements.Relative
);

In [ ]:
ChartGCData(low4DM
    , metrics: ML(Metrics.G.HctMtcp, Metrics.G.NumHeaps)
    , benchmarkFilter: Filter.RE("Stage2$")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    , iterationFilter: IntFilter.Values(0)
);

In [ ]:
ChartGCData(low4DM, Metrics.G.NumHeaps, configNameSimplifier: NameSimplifier.PrefixDashed, debug: false);

In [ ]:
var low4CompRuns = ML(("v2-fixrearranged-mult-max", "base"), ("v2-fixrearranged-mult-max-h4", "max4"),
    ("v2-fixrearranged-mult-max-svr", "svr"), ("v2-fixrearranged-mult-max-svr4", "svr4"),
    ("v2-fixrearranged-mult-max-mult8", "mult8"), ("v2-fixrearranged-mult-max-mult32", "mult32"),
    ("v2-fixrearranged-mult-max-mult8x10", "m8x10"), ("v2-fixrearranged-mult-max-mult32x10", "m32x10"),
    ("v2-fixrearranged-mult-max-x10", "x10"));

ChartGCData(low4DM, Metrics.G.HctMtcp
    , configFilter: Filter.ExcludeRE("svr")
    , configNameSimplifier: new ListSimplifier(low4CompRuns)
    );

In [ ]:
ChartGCData(
    sdsDM
    , metrics: ML(Metrics.G.AllocRateMBSec, Metrics.G.PauseDuration)
    //, benchmarkFilter: Filter.RE("Run32")
    , configNameSimplifier: NameSimplifier.PrefixDashed
);

In [ ]:
ChartGCData(
    cardsDM
    , metrics: ML(Metrics.G.AllocRateMBSec, Metrics.G.PauseDuration)
    , configFilter: Filter.RE("Run32")
);

In [ ]:
ChartGCData(
    cardsDM,
    metrics: ML(Metrics.G.PauseDuration.WithCap(100), Metrics.G.PauseStack.WithCap(100), Metrics.G.PauseFQ, Metrics.G.PauseHandles.WithCap(100), Metrics.G.PauseCards, Metrics.G.Suspend,
        new Metric<TraceGC>(gc => gc.HeapStats.GCHandleCount, "GC Handles", "#"),
        new Metric<TraceGC>(gc => gc.HeapStats.FinalizationPromotedCount, "F promoted", "#"))
    //, dataFilter: gc => gc.Generation == 0
    , configFilter: Filter.RE("Only")
    , xMetric: Metrics.X.GCIndex
    , configNameSimplifier: new ListSimplifier(("2401310010004275", "a"))
    );

In [ ]:
ChartGCData(rc3DataManager, Metrics.G.HeapSizeBefore, benchmarkFilter: Filter.RE("Stage2$"));

In [ ]:
ChartGCData(rc3DataManager, Metrics.G.NumHeaps
    , benchmarkFilter: Filter.Names("Fortunes", "FortunesDapper", "JsonHttpsHttpSys", "PlaintextQueryString", "Stage1", "Stage2", "PlaintextMvc")
    , benchmarkMap: x => (x == "Stage1" || x == "Stage2" ? "S1/2" : x));

In [ ]:
var rc3RearrNoBaseRuns = ML(("v2-rc3", "rc3"), ("v2-fixrearranged", "rc3rearr"), ("v2-tune", "rc3tune"));

ChartGCData(rc3DataManager, Metrics.G.NumHeaps
    , configNameSimplifier: new ListSimplifier(rc3RearrNoBaseRuns)
    , benchmarkFilter: new Filter(includeNames: scoutList2)
);

## Obsolete stuff - for temporary reference

In [ ]:
// Old comparison/summary code (commented out)
    /*
    public LoadInfo GetComparison(LoadInfo baseline, LoadInfo comparand)
    {
        return new LoadInfo
        {
            MaxWorkingSetMB    = DeltaPercent(baseline.MaxWorkingSetMB, comparand.MaxWorkingSetMB),
            P99WorkingSetMB = DeltaPercent(baseline.P99WorkingSetMB, comparand.P99WorkingSetMB),
            P95WorkingSetMB    = DeltaPercent(baseline.P95WorkingSetMB, comparand.P95WorkingSetMB),
            P90WorkingSetMB    = DeltaPercent(baseline.P90WorkingSetMB, comparand.P90WorkingSetMB),
            P75WorkingSetMB    = DeltaPercent(baseline.P75WorkingSetMB, comparand.P75WorkingSetMB),
            P50WorkingSetMB    = DeltaPercent(baseline.P50WorkingSetMB, comparand.P50WorkingSetMB),

            MaxPrivateMemoryMB = DeltaPercent(baseline.MaxPrivateMemoryMB, comparand.MaxPrivateMemoryMB),
            P99PrivateMemoryMB = DeltaPercent(baseline.P99PrivateMemoryMB, comparand.P99PrivateMemoryMB),
            P95PrivateMemoryMB    = DeltaPercent(baseline.P95PrivateMemoryMB, comparand.P95PrivateMemoryMB),
            P90PrivateMemoryMB    = DeltaPercent(baseline.P90PrivateMemoryMB, comparand.P90PrivateMemoryMB),
            P75PrivateMemoryMB    = DeltaPercent(baseline.P75PrivateMemoryMB, comparand.P75PrivateMemoryMB),
            P50PrivateMemoryMB    = DeltaPercent(baseline.P50PrivateMemoryMB, comparand.P50PrivateMemoryMB),
            
            Latency50thMS   = DeltaPercent(baseline.Latency50thMS, comparand.Latency50thMS),
            Latency75thMS   = DeltaPercent(baseline.Latency75thMS, comparand.Latency75thMS),
            Latency90thMS   = DeltaPercent(baseline.Latency90thMS, comparand.Latency90thMS), 
            Latency99thMS   = DeltaPercent(baseline.Latency99thMS, comparand.Latency99thMS),  
            MeanLatencyMS   = DeltaPercent(baseline.MeanLatencyMS, comparand.MeanLatencyMS),
            RequestsPerMSec = DeltaPercent(baseline.RequestsPerMSec, comparand.RequestsPerMSec),
            TotalSuspensionTimeMSec = DeltaPercent(baseline.TotalSuspensionTimeMSec, comparand.TotalSuspensionTimeMSec),
            PercentPauseTimeInGC = DeltaPercent(baseline.PercentPauseTimeInGC, comparand.PercentPauseTimeInGC),
            PercentTimeInGC = DeltaPercent(baseline.PercentTimeInGC, comparand.PercentTimeInGC),
            MeanHeapSizeBeforeMB = DeltaPercent(baseline.MeanHeapSizeBeforeMB, comparand.MeanHeapSizeBeforeMB),
            MaxHeapSizeMB = DeltaPercent(baseline.MaxHeapSizeMB, comparand.MaxHeapSizeMB),
            TotalAllocationsMB = DeltaPercent(baseline.TotalAllocationsMB, comparand.TotalAllocationsMB),
            GCScore         = DeltaPercent(baseline.GCScore, comparand.GCScore),
            MaxHeapCount = DeltaPercent(baseline.MaxHeapCount, comparand.MaxHeapCount),
            NumberOfHeapCountSwitches = DeltaPercent(baseline.NumberOfHeapCountSwitches, comparand.NumberOfHeapCountSwitches),
            NumberOfHeapCountDirectionChanges = DeltaPercent(baseline.NumberOfHeapCountDirectionChanges, comparand.NumberOfHeapCountDirectionChanges),
            Data = baseline.Data,
            Data2 = comparand.Data,
            Run = $"{baseline.Run} vs. {comparand.Run}",
            Benchmark = baseline.Benchmark,
            Id = $"{baseline.Run} vs. {comparand.Run} for {baseline.Benchmark}"
        };
    }

    public Dictionary<string, LoadInfo>? GetAllBenchmarksForRun(string run)
    {
        if (!_runToBenchmarkData.TryGetValue(run, out var benchmarksForRun))
        {
            Console.WriteLine($"No benchmarks found for run: {run}");
            return null;
        }

        return benchmarksForRun;
    }

    public void SaveBenchmarkData(string outputPath = "")
    {
        if (string.IsNullOrEmpty(outputPath))
        {
            outputPath = _basePath;
        }

        StringBuilder sb = new();
        sb.AppendLine($"Run,Benchmark,Max Working Set (MB), Max Private Memory (MB), Request/MSec, Mean Latency (MSec), Latency 50th Percentile MSec, Latency 75th Percentile MSec, Latency 90th Percentile MSec, Latency 99th Percentile MSec");
        foreach (var b in _data)
        {
            var val = b.Value; 
            sb.AppendLine($"{val.Run},{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS}");
        }

        File.WriteAllText(Path.Combine(outputPath, "AllBenchmarks.csv"), sb.ToString());
    }

    public Dictionary<string, LoadInfo>? GetAllRunsForBenchmark(string benchmark)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runsForBenchmark))
        {
            Console.WriteLine($"No runs found for benchmark: {benchmark}");
            return null;
        }

        return runsForBenchmark;
    }
*/

/*
    public LoadInfo? GetBenchmarkData(string benchmark, string run)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runData))
        {
            Console.WriteLine($"Benchmark: {benchmark} not found!");
            return null;
        }

        if (!runData.TryGetValue(run, out var loadInfo))
        {
            Console.WriteLine($"Run: {run} not found!");
            return null;
        }

        return loadInfo;
    }
*/

    /*
    public Dictionary<string, LoadInfo> GetBenchmarkToComparison(string baselineRun, string comparandRun)
    {
        Dictionary<string, LoadInfo> comparisons = new();

        Dictionary<string, LoadInfo> baselineData = new();
        Dictionary<string, LoadInfo> comparandData = new();
        HashSet<string> allBenchmarks = new();

        foreach (var d in _data)
        {
            allBenchmarks.Add(d.Value.Benchmark);

            string run = d.Key.Split("|", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries)[0];

            if (string.CompareOrdinal(run, baselineRun) == 0 && !baselineData.TryGetValue(d.Key, out var baselineInfo))
            {
                baselineInfo = baselineData[d.Value.Benchmark] = d.Value;
            }

            else if (string.CompareOrdinal(run, comparandRun) == 0 && !comparandData.TryGetValue(d.Key, out var comparandInfo))
            {
                comparandInfo = comparandData[d.Value.Benchmark] = d.Value;
            }
        }

        foreach (var benchmark in allBenchmarks)
        {
            if (!baselineData.TryGetValue(benchmark, out var baselineBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the baseline: {baselineRun}");
                continue;
            }

            if (!comparandData.TryGetValue(benchmark, out var comparandBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the comparand: {comparandRun}");
                continue;
            }

            LoadInfo comparison = GetComparison(baselineBenchmarkInfo, comparandBenchmarkInfo);
            comparisons[benchmark] = comparison;
        }
        
        return comparisons;
    }
    */

    // Haven't used this in a while - writes a summary file to disk

/*
    public void SummarizeResults(DataManager dataManager, string outFile, Dictionary<string, LoadInfo> info = null)
    {
        if (info == null)
        {
            info = dataManager._data;
        }

        using (StreamWriter sw = new StreamWriter(outFile))
    {
        sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |", 
                        "run", "benchmark", "gen0", "pause", "gen1", "pause", "ngc2", "pause", "bgc", "pause", "allocMB", "alloc/gc", "pct", "peakMB", "meanMB");
        sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |", 
                        "", "", "", "susp", "", "susp", "", "susp", "", "susp", "", "", "", "totalcpu", "meancpu");
        sw.WriteLine("{0}", new String('-', 174));
        foreach (var kvp in info)
        {
            List<TraceGC> gcs = kvp.Value?.Data?.GCs;
            if (gcs == null || gcs.Count == 0)
            {
                continue;
            }

            int[] gc_counts = new int[4];
            double[] gc_pauses = new double[4];
            double[] gc_susps = new double[4];
            for (int i = 0; i < gcs.Count; i++)
            {
                TraceGC gc = gcs[i];
                //if (gc.SuspendDurationMSec > 5) sw.WriteLine($"i={gc.Number} gen={gc.Generation} suspension={gc.SuspendDurationMSec} totalpause={gc.PauseDurationMSec}");
                if (gc.Generation < 2)
                {
                    gc_counts[gc.Generation]++;
                    gc_pauses[gc.Generation] += gc.PauseDurationMSec;
                    gc_susps[gc.Generation] += gc.SuspendDurationMSec;
                }
                else
                {
                    if (gc.Type == GCType.BackgroundGC)
                    {
                        gc_counts[3]++;
                        gc_pauses[3] += gc.PauseDurationMSec;
                        gc_susps[3] += gc.SuspendDurationMSec;
                    }
                    else
                    {
                        gc_counts[2]++;
                        gc_pauses[2] += gc.PauseDurationMSec;
                        gc_susps[2] += gc.SuspendDurationMSec;
                    }
                }
            }
            
            for (int i = 0; i < 4; i++)
            {
                if (gc_counts[i] > 0)
                {
                    gc_pauses[i] /= gc_counts[i];
                    gc_susps[i] /= gc_counts[i];
                }
            }
            
            sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |",
                kvp.Value.Run, kvp.Value.Benchmark, gc_counts[0], gc_pauses[0], gc_counts[1], gc_pauses[1], gc_counts[2], gc_pauses[2], gc_counts[3], gc_pauses[3],
                kvp.Value.Data.Stats.TotalAllocatedMB, (kvp.Value.Data.Stats.TotalAllocatedMB / gcs.Count), kvp.Value.Data.Stats.GetGCPauseTimePercentage(), kvp.Value.Data.Stats.MaxSizePeakMB, kvp.Value.Data.Stats.MeanSizePeakMB);
            sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |",
                "", "", "", gc_susps[0], "", gc_susps[1], "", gc_susps[2], "", gc_susps[3],
                "", "", "", kvp.Value.Data.Stats.TotalCpuMSec, kvp.Value.Data.Stats.MeanCpuMSec);
        }
    }}

class MeanDataComparison
{
    public string bench { get; set; }
    public double baselineMaxPrivateMemoryMB { get; set; }
    public double baselineP50PrivateMemoryMB { get; set; }
    public double baselineRequestsPerMSec { get; set; }
    public double avgMaxPrivateMemoryMBDiff { get; set; }
    public double avgP50PrivateMemoryMBDiff { get; set; }
    public double avgRequestsPerMSecDiff { get; set; }
    public double baselineCVMaxPrivateMemoryMB { get; set; }
    public double baselineCVP50PrivateMemoryMB { get; set; }
    public double baselineCVRequestsPerMSec { get; set; }
    public double fixCVMaxPrivateMemoryMB { get; set; }
    public double fixCVP50PrivateMemoryMB { get; set; }
    public double fixCVRequestsPerMSec { get; set; }
    public double cvMaxPrivateMemoryMBDiff { get; set; }
    public double cvP50PrivateMemoryMBDiff { get; set; }
    public double cvRequestsPerMSecDiff { get; set; }
}

double GetCV(List<double> dataPoints, out double avg)
{
    // for (int i = 0; i < dataPoints.Count; i++)
    // {
    //     Console.WriteLine("item {0}: {1}", i, dataPoints[i]);
    // }
    double mean = dataPoints.Average();
    avg = mean;
    double sumOfSquaredDifferences = dataPoints.Sum(val => Math.Pow(val - mean, 2));
    double populationStandardDeviation = Math.Sqrt(sumOfSquaredDifferences / dataPoints.Count);
    double coefficientOfVariation = (populationStandardDeviation / mean) * 100;
    return coefficientOfVariation;
}

// accommodates when there are different numbers of iterations in first and second run.
// returns a list of benchmarks we added to the comparison data
List<MeanDataComparison> SummarizeResultsByBench(DataManager dataManager, List<string> runNames, string benchName = null)
{
    Dictionary<string, Dictionary<string, LoadInfo>> benchmarkToRunData = dataManager._benchmarkToRunData;
    Console.WriteLine("benchmarkToRunData has {0} tests\n", benchmarkToRunData.Count);

    //bool fLogDetail = false;
    bool fLogDetail = true;

    string strSeparator = new String('-', 223);
    Console.WriteLine("{0}", strSeparator);

    // key is the name of the run, eg, "baseline" or "fix". For each run, we add its summary data to a list.
    Dictionary<string, List<BenchmarkSummaryData>> summaryDataForRuns = new Dictionary<string, List<BenchmarkSummaryData>>(2);
    List<MeanDataComparison> comparisonData = new List<MeanDataComparison>(51);

    foreach (var benchmarkData in benchmarkToRunData)
    {
        // // Console.WriteLine("benchmark is {0}", benchmarkData.Key);

        if ((benchName == null) || benchmarkData.Key.Equals(benchName, StringComparison.OrdinalIgnoreCase))
        {
            summaryDataForRuns.Clear();

            if (fLogDetail)
            {
                Console.WriteLine("{0,25} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 8:0.00} | {14, 8:0.00} | {15, 8:0.00} | {16, 8:0.00} | {17, 8:0.00} | {18, 4:0.00} | {19, 10} |",
                                "run", "benchmark", "gen0", "pause", "gen1", "pause", "ngc2", "pause", "bgc", "pause", "allocMB", "alloc/gc", "pct", "peakMB", "meanMB", "max mem", "rps", "latency", "hc", "gc count");
                Console.WriteLine("{0}", strSeparator);                                
            }

            // if no runs observed an hc change, we don't keep it in the summary data.
            int totalHCChanges = 0;

            foreach (var kvp in benchmarkData.Value)
            {
                List<TraceGC> gcs = kvp.Value?.Data?.GCs;
                // We don't look at benchmarks that did very few GCs
                if ((gcs == null) || (gcs.Count == 0))
                {
                    continue;
                }

                int[] gc_counts = new int[4];
                double[] gc_pauses = new double[4];
                for (int i = 0; i < gcs.Count; i++)
                {
                    TraceGC gc = gcs[i];
                    if (gc.Generation < 2)
                    {
                        gc_counts[gc.Generation]++;
                        gc_pauses[gc.Generation] += gc.PauseDurationMSec;
                    }
                    else
                    {
                        if (gc.Type == GCType.BackgroundGC)
                        {
                            gc_counts[3]++;
                            gc_pauses[3] += gc.PauseDurationMSec;
                        }
                        else
                        {
                            gc_counts[2]++;
                            gc_pauses[2] += gc.PauseDurationMSec;
                        }
                    }
                }

                for (int i = 0; i < 4; i++)
                {
                    if (gc_counts[i] > 0)
                    {
                        gc_pauses[i] /= gc_counts[i];
                    }
                }

                if (fLogDetail)
                {
                    Console.WriteLine("{0,25} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 8:0.00} | {14, 8:0.00} | {15, 8:0.00} | {16, 8:0.00} | {17, 8:0.00} | {18, 4} | {19, 10} |",
                        kvp.Value.Run, kvp.Value.Benchmark, gc_counts[0], gc_pauses[0], gc_counts[1], gc_pauses[1], gc_counts[2], gc_pauses[2], gc_counts[3], gc_pauses[3],
                        kvp.Value.Data.Stats.TotalAllocatedMB, (kvp.Value.Data.Stats.TotalAllocatedMB / gcs.Count), kvp.Value.Data.Stats.GetGCPauseTimePercentage(), kvp.Value.Data.Stats.MaxSizePeakMB, kvp.Value.Data.Stats.MeanSizePeakMB,
                        kvp.Value.MaxPrivateMemoryMB, kvp.Value.RequestsPerMSec, kvp.Value.MeanLatencyMS, kvp.Value.NumberOfHeapCountSwitches, kvp.Value.Data.Stats.Count);
                }

                totalHCChanges += (int)kvp.Value.NumberOfHeapCountSwitches;

                for (int runIdx = 0; runIdx < runNames.Count; runIdx++)
                {
                    if (kvp.Value.Run.StartsWith(runNames[runIdx]))
                    {
                        BenchmarkSummaryData data = new BenchmarkSummaryData 
                        {
                            MaxPrivateMemoryMB = kvp.Value.MaxPrivateMemoryMB,
                            P50PrivateMemoryMB = kvp.Value.P50PrivateMemoryMB,
                            RequestsPerMSec = kvp.Value.RequestsPerMSec,
                        };

                        if (summaryDataForRuns.ContainsKey(runNames[runIdx]))
                        {
                            summaryDataForRuns[runNames[runIdx]].Add(data);
                        }
                        else
                        {
                            List<BenchmarkSummaryData> listData = new List<BenchmarkSummaryData>(3);
                            listData.Add(data);
                            summaryDataForRuns.Add(runNames[runIdx], listData);
                        }
                        break;
                    }
                }
            }

            if (fLogDetail)
            {
                Console.WriteLine("{0}", strSeparator);
            }

            if (totalHCChanges == 0)
            {
                //Console.WriteLine("don't do comparison for bench {0}! no HC changes", benchmarkData.Key);
                continue;
            }

            // Now write some summary stuff
            //Console.WriteLine("summary dictionary has {0} elements", summaryDataForRuns.Count);

            if (fLogDetail)
            {
                Console.WriteLine("{0,45} | {1,10} | {2,10} | {3,10} | {4,10} | {5,10} | {6,10} |", "data", "max mem", "CV%", "p50 mem", "CV%", "rps", "CV%");
            }

            int numRuns = summaryDataForRuns.Count;
            double[] avgMaxPrivateMemoryMBForRuns = new double [numRuns];
            double[] avgP50PrivateMemoryMBForRuns = new double [numRuns];
            double[] avgRequestsPerMSecForRuns = new double [numRuns];
            double[] cvMaxPrivateMemoryMBForRuns = new double [numRuns];
            double[] cvP50PrivateMemoryMBForRuns = new double [numRuns];
            double[] cvRequestsPerMSecForRuns = new double [numRuns];

            for (int i = 0; i < summaryDataForRuns.Count; i++)
            {
                //Console.WriteLine("bench {0} has {1} iteration in run {2}", benchmarkData.Key, summaryDataForRuns.ElementAt(i).Value.Count, summaryDataForRuns.ElementAt(i).Key);
                List<BenchmarkSummaryData> listData = summaryDataForRuns.ElementAt(i).Value;

                // for (int runIdx  = 0; runIdx < listData.Count; runIdx++)
                // {
                //     Console.WriteLine("run {0} iter {1} max mem {2}, rps {3}", summaryDataForRuns.ElementAt(i).Key, runIdx, listData[runIdx].MaxPrivateMemoryMB, listData[runIdx].RequestsPerMSec);
                // }
                List<double> listMaxPrivateMemoryMB = listData.Select(s => s.MaxPrivateMemoryMB).ToList();
                double avgMaxPrivateMemoryMB, avgP50PrivateMemoryMB, avgRequestsPerMSec;
                double cvMaxPrivateMemoryMB = GetCV(listMaxPrivateMemoryMB, out avgMaxPrivateMemoryMB);
                List<double> listP50PrivateMemoryMB = listData.Select(s => s.P50PrivateMemoryMB).ToList();
                double cvP50PrivateMemoryMB = GetCV(listP50PrivateMemoryMB, out avgP50PrivateMemoryMB);
                List<double> listRequestsPerMSec = listData.Select(s => s.RequestsPerMSec).ToList();
                double cvRequestsPerMSec = GetCV(listRequestsPerMSec, out avgRequestsPerMSec);

                avgMaxPrivateMemoryMBForRuns[i] = avgMaxPrivateMemoryMB;
                avgP50PrivateMemoryMBForRuns[i] = avgP50PrivateMemoryMB;
                avgRequestsPerMSecForRuns[i] = avgRequestsPerMSec;
                cvMaxPrivateMemoryMBForRuns[i] = cvMaxPrivateMemoryMB;
                cvP50PrivateMemoryMBForRuns[i] = cvP50PrivateMemoryMB;
                cvRequestsPerMSecForRuns[i] = cvRequestsPerMSec;
                
                if (fLogDetail)
                {
                    Console.WriteLine("{0,45} | {1,10:0.00} | {2,10:0.00} | {3,10:0.00} | {4,10:0.00} | {5,10:0.00} | {6,10:0.00} |", ("-" + benchmarkData.Key + "-" + summaryDataForRuns.ElementAt(i).Key), 
                        avgMaxPrivateMemoryMB, cvMaxPrivateMemoryMB, avgP50PrivateMemoryMB, cvP50PrivateMemoryMB, avgRequestsPerMSec, cvRequestsPerMSec);
                }
            }

            // I'm just assuming we only have 2 runs.
            MeanDataComparison comp = new MeanDataComparison
            {
                bench = benchmarkData.Key,
                baselineMaxPrivateMemoryMB = avgMaxPrivateMemoryMBForRuns[0],
                baselineP50PrivateMemoryMB = avgP50PrivateMemoryMBForRuns[0],
                baselineRequestsPerMSec = avgRequestsPerMSecForRuns[0],
                avgMaxPrivateMemoryMBDiff = (avgMaxPrivateMemoryMBForRuns[1] - avgMaxPrivateMemoryMBForRuns[0]) * 100.0 / avgMaxPrivateMemoryMBForRuns[0],
                avgP50PrivateMemoryMBDiff = (avgP50PrivateMemoryMBForRuns[1] - avgP50PrivateMemoryMBForRuns[0]) * 100.0 / avgP50PrivateMemoryMBForRuns[0],
                avgRequestsPerMSecDiff = (avgRequestsPerMSecForRuns[1] - avgRequestsPerMSecForRuns[0]) * 100.0 / avgRequestsPerMSecForRuns[0],
                baselineCVMaxPrivateMemoryMB = cvMaxPrivateMemoryMBForRuns[0],
                baselineCVP50PrivateMemoryMB = cvP50PrivateMemoryMBForRuns[0],
                baselineCVRequestsPerMSec = cvRequestsPerMSecForRuns[0],
                fixCVMaxPrivateMemoryMB = cvMaxPrivateMemoryMBForRuns[1],
                fixCVP50PrivateMemoryMB = cvP50PrivateMemoryMBForRuns[1],
                fixCVRequestsPerMSec = cvRequestsPerMSecForRuns[1],
                cvMaxPrivateMemoryMBDiff = (cvMaxPrivateMemoryMBForRuns[1] - cvMaxPrivateMemoryMBForRuns[0]) * 100.0 / cvMaxPrivateMemoryMBForRuns[0],
                cvP50PrivateMemoryMBDiff = (cvP50PrivateMemoryMBForRuns[1] - cvP50PrivateMemoryMBForRuns[0]) * 100.0 / cvP50PrivateMemoryMBForRuns[0],
                cvRequestsPerMSecDiff = (cvRequestsPerMSecForRuns[1] - cvRequestsPerMSecForRuns[0]) * 100.0 / cvRequestsPerMSecForRuns[0],                
            };
            comparisonData.Add(comp);

            if (fLogDetail)
            {
                Console.WriteLine("{0}\n", strSeparator);
            }

            if (benchName != null)
            {
                break;
            }
        }
    }

    if (true)
    {
        Console.WriteLine("displaying {0} benches that observed HC changes", comparisonData.Count);

        Console.WriteLine("{0,35} | {1, 9} | {2,9} | {3,9} | {4,9} | {5,9} | {6,9} | {7,9} | {8,9} | {9,9} | {10,9} | {11,9} | {12,9} | {13,9} | {14,9} | {15,9} |",
            "bench", "b max mem", "max mem %", "b cv%", "f cv%", "CV% %", "b p50 mem", "p50 mem %", "b cv%", "f cv%", "CV% %", "b rps", "rps %", "b cv%", "f cv%", "CV% %");

        var sortedComparisonData = comparisonData.OrderByDescending(a => a.cvMaxPrivateMemoryMBDiff).ToList();
        //var sortedComparisonData = comparisonData.OrderBy(a => a.avgMaxPrivateMemoryMBDiff).ToList();
        for (int benchIdx = 0; benchIdx < sortedComparisonData.Count; benchIdx++)
        {
            MeanDataComparison currentComp = sortedComparisonData[benchIdx];
            Console.WriteLine("{0,35} | {1,9:0.00} | {2,9:0.00} | {3,9:0.00} | {4,9:0.00} | {5,9:0.00} | {6,9:0.00} | {7,9:0.00} | {8,9:0.00} | {9,9:0.00} | {10,9:0.00} | {11,9:0.00} | {12,9:0.00} | {13,9:0.00} | {14,9:0.00} | {15,9:0.00} |",
                currentComp.bench,
                currentComp.baselineMaxPrivateMemoryMB, currentComp.avgMaxPrivateMemoryMBDiff, currentComp.baselineCVMaxPrivateMemoryMB, currentComp.fixCVMaxPrivateMemoryMB, currentComp.cvMaxPrivateMemoryMBDiff,
                currentComp.baselineP50PrivateMemoryMB, currentComp.avgP50PrivateMemoryMBDiff, currentComp.baselineCVP50PrivateMemoryMB, currentComp.fixCVP50PrivateMemoryMB, currentComp.cvP50PrivateMemoryMBDiff,
                currentComp.baselineRequestsPerMSec, currentComp.avgRequestsPerMSecDiff, currentComp.baselineCVRequestsPerMSec, currentComp.fixCVRequestsPerMSec, currentComp.cvRequestsPerMSecDiff);
        }
    }

    return comparisonData;
}
*/

// I haven't used this in a while.  I'm not sure if it works.
/*
    public void SaveDifferences(DataManager dataManager, string baseline, string comparand, List<string> sortingCriteria = null)
    {
        // This function assumes the runs are all in:
        // {build}_{iteration} form.
        // Else, it will except.
using (StreamWriter sw = new StreamWriter(@"c:\home\repro\hc\hc-savediff.txt")) {
    sw.WriteLine("start");
        // Iteration -> LoadInfos
        Dictionary<int, List<LoadInfo>> iterationData = new();

        // Get the max iteration.
        int maxIteration = -1;
        foreach (var run in dataManager._runToBenchmarkData)
        {
            string runName = run.Key;
            int iteration = 0;
            if (run.Key.Contains("_"))
            {
                string[] split = run.Key.Split("_");
                Debug.Assert(split.Length == 2);
                string build = split[0];
                string iterationAsString = split[1];
                iteration = Convert.ToInt32(iterationAsString);
            }
            maxIteration = System.Math.Max(iteration, maxIteration);
        }
        sw.WriteLine(maxIteration);
        // Compute Average Diff
        // Build to Benchmark -> Data
        Dictionary<string, Dictionary<string, LoadInfo>> averageData = new();

        for (int i = 0; i <= maxIteration; i++)
        {
            sw.WriteLine(i);
            sw.WriteLine(maxIteration);
            string baselineIteration;
            string comparandIteration;
            if (maxIteration == 0)
            {
                baselineIteration = baseline;
                comparandIteration = comparand;
            }
            else
            {
                baselineIteration = baseline  + "_" + i.ToString();
                comparandIteration = comparand + "_" + i.ToString();
            }
            foreach (var x in dataManager._runToBenchmarkData.Keys) { sw.WriteLine(x); }
            Dictionary<string, LoadInfo> baselineIterationRuns  = dataManager._runToBenchmarkData[baselineIteration];
            Dictionary<string, LoadInfo> comparandIterationRuns = dataManager._runToBenchmarkData[comparandIteration];

            foreach (var b in baselineIterationRuns)
            {
                if (!iterationData.TryGetValue(i, out var benchmarks))
                {
                    iterationData[i] = benchmarks = new();
                }

                benchmarks.Add(dataManager.GetComparison(baselineIterationRuns[b.Key], comparandIterationRuns[b.Key]));
            }

            if (!averageData.TryGetValue(baseline, out var bVal))
            {
                averageData[baseline] = bVal = new();
                foreach (var benchmark in baselineIterationRuns)
                {
                    bVal[benchmark.Key] = new LoadInfo
                    {
                        Benchmark = benchmark.Key,
                        MaxWorkingSetMB    = benchmark.Value.MaxWorkingSetMB,
                        MaxPrivateMemoryMB = benchmark.Value.MaxPrivateMemoryMB,
                        P99PrivateMemoryMB = benchmark.Value.P99PrivateMemoryMB,
                        P95PrivateMemoryMB = benchmark.Value.P95PrivateMemoryMB,
                        P90PrivateMemoryMB = benchmark.Value.P90PrivateMemoryMB,
                        P75PrivateMemoryMB = benchmark.Value.P75PrivateMemoryMB,
                        P50PrivateMemoryMB = benchmark.Value.P50PrivateMemoryMB,
                        RequestsPerMSec = benchmark.Value.RequestsPerMSec,
                        MeanLatencyMS   = benchmark.Value.MeanLatencyMS,
                        Latency50thMS   = benchmark.Value.Latency50thMS, 
                        Latency75thMS   = benchmark.Value.Latency75thMS,
                        Latency90thMS   = benchmark.Value.Latency90thMS,
                        Latency99thMS   = benchmark.Value.Latency99thMS,
                        MaxHeapCount = benchmark.Value.MaxHeapCount,
                        NumberOfHeapCountSwitches = benchmark.Value.NumberOfHeapCountSwitches,
                        NumberOfHeapCountDirectionChanges = benchmark.Value.NumberOfHeapCountDirectionChanges,
                    };
                }
            }

            else
            {
                foreach (var benchmark in baselineIterationRuns)
                {
                    var data = bVal[benchmark.Key];
                    data.Benchmark = benchmark.Key;
                    data.MaxWorkingSetMB    += benchmark.Value.MaxWorkingSetMB;
                    data.MaxPrivateMemoryMB += benchmark.Value.MaxPrivateMemoryMB;
                    data.P99PrivateMemoryMB += benchmark.Value.P99PrivateMemoryMB;
                    data.P95PrivateMemoryMB += benchmark.Value.P95PrivateMemoryMB;
                    data.P90PrivateMemoryMB += benchmark.Value.P90PrivateMemoryMB;
                    data.P75PrivateMemoryMB += benchmark.Value.P75PrivateMemoryMB;
                    data.P50PrivateMemoryMB += benchmark.Value.P50PrivateMemoryMB;
                    data.RequestsPerMSec += benchmark.Value.RequestsPerMSec;
                    data.MeanLatencyMS   += benchmark.Value.MeanLatencyMS;
                    data.Latency50thMS   += benchmark.Value.Latency50thMS; 
                    data.Latency75thMS   += benchmark.Value.Latency75thMS;
                    data.Latency90thMS   += benchmark.Value.Latency90thMS;
                    data.Latency99thMS   += benchmark.Value.Latency99thMS;
                    data.MaxHeapCount += benchmark.Value.MaxHeapCount;
                    data.NumberOfHeapCountSwitches += benchmark.Value.NumberOfHeapCountSwitches;
                    data.NumberOfHeapCountDirectionChanges += benchmark.Value.NumberOfHeapCountDirectionChanges;
                }
            }

            if (!averageData.TryGetValue(comparand, out var cVal))
            {
                averageData[comparand] = cVal = new();
                foreach (var benchmark in comparandIterationRuns)
                {
                    cVal[benchmark.Key] = new LoadInfo
                    {
                        Benchmark = benchmark.Key,
                        MaxWorkingSetMB    = benchmark.Value.MaxWorkingSetMB,
                        MaxPrivateMemoryMB = benchmark.Value.MaxPrivateMemoryMB,
                        P99PrivateMemoryMB = benchmark.Value.P99PrivateMemoryMB,
                        P95PrivateMemoryMB = benchmark.Value.P95PrivateMemoryMB,
                        P90PrivateMemoryMB = benchmark.Value.P90PrivateMemoryMB,
                        P75PrivateMemoryMB = benchmark.Value.P75PrivateMemoryMB,
                        P50PrivateMemoryMB = benchmark.Value.P50PrivateMemoryMB,
                        RequestsPerMSec = benchmark.Value.RequestsPerMSec,
                        MeanLatencyMS   = benchmark.Value.MeanLatencyMS,
                        Latency50thMS   = benchmark.Value.Latency50thMS, 
                        Latency75thMS   = benchmark.Value.Latency75thMS,
                        Latency90thMS   = benchmark.Value.Latency90thMS,
                        Latency99thMS   = benchmark.Value.Latency99thMS,
                        MaxHeapCount    = benchmark.Value.MaxHeapCount,
                        NumberOfHeapCountSwitches = benchmark.Value.NumberOfHeapCountSwitches,
                        NumberOfHeapCountDirectionChanges = benchmark.Value.NumberOfHeapCountDirectionChanges,
                    };
                }
            }

            else
            {
                foreach (var benchmark in comparandIterationRuns)
                {
                    var data = cVal[benchmark.Key];
                    data.Benchmark = benchmark.Key;
                    data.MaxWorkingSetMB    += benchmark.Value.MaxWorkingSetMB;
                    data.MaxPrivateMemoryMB += benchmark.Value.MaxPrivateMemoryMB;
                    data.P99PrivateMemoryMB += benchmark.Value.P99PrivateMemoryMB;
                    data.P95PrivateMemoryMB += benchmark.Value.P95PrivateMemoryMB;
                    data.P90PrivateMemoryMB += benchmark.Value.P90PrivateMemoryMB;
                    data.P75PrivateMemoryMB += benchmark.Value.P75PrivateMemoryMB;
                    data.P50PrivateMemoryMB += benchmark.Value.P50PrivateMemoryMB;
                    data.RequestsPerMSec += benchmark.Value.RequestsPerMSec;
                    data.MeanLatencyMS   += benchmark.Value.MeanLatencyMS;
                    data.Latency50thMS   += benchmark.Value.Latency50thMS; 
                    data.Latency75thMS   += benchmark.Value.Latency75thMS;
                    data.Latency90thMS   += benchmark.Value.Latency90thMS;
                    data.Latency99thMS   += benchmark.Value.Latency99thMS;
                    data.MaxHeapCount    += benchmark.Value.MaxHeapCount;
                    data.NumberOfHeapCountSwitches += benchmark.Value.NumberOfHeapCountSwitches;
                    data.NumberOfHeapCountDirectionChanges += benchmark.Value.NumberOfHeapCountDirectionChanges;
                }
            }
        }

        foreach (var benchmark in dataManager._benchmarkToRunData)
        {
            foreach (var build in averageData)
            {
                var data = build.Value[benchmark.Key];
                data.Benchmark = benchmark.Key;
                data.MaxWorkingSetMB    /= (maxIteration + 1); 
                data.MaxPrivateMemoryMB /=  (maxIteration + 1);
                data.P99PrivateMemoryMB /=  (maxIteration + 1);
                data.P95PrivateMemoryMB /=  (maxIteration + 1);
                data.P90PrivateMemoryMB /=  (maxIteration + 1);
                data.P75PrivateMemoryMB /=  (maxIteration + 1);
                data.P50PrivateMemoryMB /=  (maxIteration + 1);
                data.RequestsPerMSec /=  (maxIteration + 1);
                data.MeanLatencyMS   /= (maxIteration + 1);
                data.Latency50thMS   /= (maxIteration + 1);
                data.Latency75thMS   /= (maxIteration + 1);
                data.Latency90thMS   /= (maxIteration + 1);
                data.Latency99thMS   /= (maxIteration + 1);
                data.MaxHeapCount /= (maxIteration + 1);
                data.NumberOfHeapCountSwitches /= (maxIteration + 1);
                data.NumberOfHeapCountDirectionChanges /= (maxIteration + 1);
            }
        }

        string DisplayDetailsForABenchmark(LoadInfo val) =>
            $"{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS},{val.NumberOfHeapCountSwitches},{val.MaxHeapCount}";

        if (sortingCriteria == null)
        {
            sortingCriteria = new() { nameof(LoadInfo.MaxPrivateMemoryMB) };
        }

        foreach (var s in sortingCriteria)
        {
            Func<LoadInfo, double> sortingFunctor = null;
            Func<KeyValuePair<string, LoadInfo>, double> selectionFunctor = null;

            switch (s)
            {
                case nameof(LoadInfo.MaxWorkingSetMB):
                    sortingFunctor = (data) => data.MaxWorkingSetMB;
                    selectionFunctor = (data) => data.Value.MaxWorkingSetMB;
                    break;
                case nameof(LoadInfo.MaxPrivateMemoryMB):
                    sortingFunctor = (data) => data.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P99PrivateMemoryMB):
                    sortingFunctor = (data) => data.P99PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P99PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P95PrivateMemoryMB):
                    sortingFunctor = (data) => data.P95PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P95PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P90PrivateMemoryMB):
                    sortingFunctor = (data) => data.P90PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P90PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P75PrivateMemoryMB):
                    sortingFunctor = (data) => data.P75PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P75PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P50PrivateMemoryMB):
                    sortingFunctor = (data) => data.P50PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P50PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.RequestsPerMSec):
                    sortingFunctor = (data) => data.RequestsPerMSec;
                    selectionFunctor = (data) => data.Value.RequestsPerMSec;
                    break;
                case nameof(LoadInfo.MeanLatencyMS):
                    sortingFunctor = (data) => data.MeanLatencyMS;
                    selectionFunctor = (data) => data.Value.MeanLatencyMS;
                    break;
                case nameof(LoadInfo.Latency50thMS):
                    sortingFunctor = (data) => data.Latency50thMS;
                    selectionFunctor = (data) => data.Value.Latency50thMS;
                    break;
                case nameof(LoadInfo.Latency75thMS):
                    sortingFunctor = (data) => data.Latency75thMS;
                    selectionFunctor = (data) => data.Value.Latency75thMS;
                    break;
                case nameof(LoadInfo.Latency90thMS):
                    sortingFunctor = (data) => data.Latency90thMS;
                    selectionFunctor = (data) => data.Value.Latency90thMS;
                    break;
                case nameof(LoadInfo.Latency99thMS):
                    sortingFunctor = (data) => data.Latency99thMS;
                    selectionFunctor = (data) => data.Value.Latency99thMS;
                    break;
                case nameof(LoadInfo.MaxHeapCount):
                    sortingFunctor = (data) => data.MaxHeapCount;
                    selectionFunctor = (data) => data.Value.MaxHeapCount;
                    break;
                case nameof(LoadInfo.NumberOfHeapCountSwitches):
                    sortingFunctor = (data) => data.NumberOfHeapCountSwitches;
                    selectionFunctor = (data) => data.Value.NumberOfHeapCountSwitches;
                    break;
                case nameof(LoadInfo.NumberOfHeapCountDirectionChanges):
                    sortingFunctor = (data) => data.NumberOfHeapCountDirectionChanges;
                    selectionFunctor = (data) => data.Value.NumberOfHeapCountDirectionChanges;
                    break;

                case nameof(BenchmarkSummaryData.TotalSuspensionTimeMSec):
                    sortingFunctor = (data) => data.TotalSuspensionTimeMSec;
                    selectionFunctor = (data) => data.Value.TotalSuspensionTimeMSec;
                    break;
                case nameof(BenchmarkSummaryData.PercentPauseTimeInGC):
                    sortingFunctor = (data) => data.PercentPauseTimeInGC;
                    selectionFunctor = (data) => data.Value.PercentPauseTimeInGC;
                    break;
                case nameof(BenchmarkSummaryData.PercentTimeInGC):
                    sortingFunctor = (data) => data.PercentTimeInGC;
                    selectionFunctor = (data) => data.Value.PercentTimeInGC;
                    break;
                case nameof(BenchmarkSummaryData.MeanHeapSizeBeforeMB):
                    sortingFunctor = (data) => data.MeanHeapSizeBeforeMB;
                    selectionFunctor = (data) => data.Value.MeanHeapSizeBeforeMB;
                    break;
                case nameof(BenchmarkSummaryData.MaxHeapSizeMB):
                    sortingFunctor = (data) => data.MaxHeapSizeMB;
                    selectionFunctor = (data) => data.Value.MaxHeapSizeMB;
                    break;
                case nameof(BenchmarkSummaryData.TotalAllocationsMB):
                    sortingFunctor = (data) => data.TotalAllocationsMB;
                    selectionFunctor = (data) => data.Value.TotalAllocationsMB;
                    break;
                case nameof(BenchmarkSummaryData.GCScore):
                    sortingFunctor = (data) => data.GCScore;
                    selectionFunctor = (data) => data.Value.GCScore;
                    break;

                default:
                    throw new Exception($"unexpected {s}");
            }

            List<List<LoadInfo>> sortedLoadInfo = new(); 
            foreach (var iteration in iterationData)
            {
                sortedLoadInfo.Add(iteration.Value.OrderByDescending(sortingFunctor).ToList());
            }

            List<LoadInfo> sortedAverages = new();

            foreach (var benchmark in averageData[baseline])
            {
                LoadInfo baselineInfo   = benchmark.Value;
                LoadInfo comparandInfo  = averageData[comparand][benchmark.Key];
                LoadInfo comparisonInfo = dataManager.GetComparison(baselineInfo, comparandInfo);
                sortedAverages.Add(comparisonInfo);
            }
            sortedAverages = sortedAverages.OrderByDescending(sortingFunctor).ToList();

            // Create CSV.
            StringBuilder top = new();

            // Iterate over each of the runs.
            const int singleBuildColumnSize = 11;
            int numberOfIterations = maxIteration + 1;
            string columnHeader = "Benchmark Name,WorkingSetMB,PrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS,# HC Switches";

            int totalCountOfBenchmarks = sortedLoadInfo.First().Count;

            string first = string.Join("", Enumerable.Range(0, numberOfIterations).Select(build => build + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize))));
            string second = string.Join(",,", Enumerable.Repeat(columnHeader, numberOfIterations));

            // Add the average diff.
            first  += "Average Diff %" + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize));
            second += ",," + string.Join(",,", columnHeader);

            top.AppendLine(first);
            top.AppendLine(second);

            for (int benchmarkIdx = 0; benchmarkIdx < totalCountOfBenchmarks; benchmarkIdx++)
            {
                string benchmarkData = string.Join(",,", Enumerable.Range(0, numberOfIterations).Select(iteration => DisplayDetailsForABenchmark(sortedLoadInfo[iteration][benchmarkIdx])));
                benchmarkData += $",,{DisplayDetailsForABenchmark(sortedAverages[benchmarkIdx])}";

                top.AppendLine(benchmarkData);
            }

            File.WriteAllText(Path.Combine(dataManager._basePath, $"Difference_{s}.csv"), top.ToString());

            var layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Benchmark Name" },
                yaxis = new Yaxis { title = $"{s}" },
                width = 1500,
                title = $"Raw values of {s} for Runs"
            };

            List<Scatter> scatters = new();

            const int baseColor = 150;

            for (int iterationIdx = 0; iterationIdx <= maxIteration; iterationIdx++)
            {
                string baselineIteration;
                string comparandIteration;
            if (maxIteration == 0)
            {
                baselineIteration = baseline;
                comparandIteration = comparand;
            }
            else
            {
                baselineIteration = baseline  + "_" + iterationIdx.ToString();
                comparandIteration = comparand + "_" + iterationIdx.ToString();
            }

                Dictionary<string, LoadInfo> baselineData  = dataManager._runToBenchmarkData[baselineIteration];
                Dictionary<string, LoadInfo> comparandData = dataManager._runToBenchmarkData[comparandIteration];

                if (iterationIdx == 0)
                {
                    var sortedBaseline = baselineData.Values.OrderByDescending(sortingFunctor);
                    baselineData = sortedBaseline.ToDictionary(d => d.Benchmark);
                }

                Scatter baselineScatter = new()
                {
                    x = baselineData.Select(b => b.Key),
                    y = baselineData.Select(selectionFunctor),
                    name = $"{baselineIteration} - {s}",
                    mode = "markers",
                    marker = new Marker { color = $"rgb({baseColor + iterationIdx * 50}, 0, 0)" } 
                };

                Scatter comparandScatter = new()
                {
                    x = comparandData.Select(b => b.Key),
                    y = comparandData.Select(selectionFunctor),
                    name = $"{comparandIteration} - {s}",
                    mode = "markers",
                    marker = new Marker { color = $"rgb(0, 0, {baseColor + iterationIdx * 50})" } 
                };

                scatters.Add(baselineScatter);
                scatters.Add(comparandScatter);
            }

            Chart.Plot(scatters, layout).Display();
        }
    }
    }
    */

        /*
    public class BuildNameComparer : IEqualityComparer<BuildName>
    {
        public bool Equals(BuildName b1, BuildName b2) => b1.InData == b2.InData;
        public int GetHashCode(BuildName b) => b.InData.GetHashCode();
    }
    public record PerBuildData((DataType, string) Criteria, string Unit, BuildName BuildName, Func<BenchmarkSummaryData, double> Selector, List<BenchmarkSummaryData> Data);
    
    public Func<List<double>, double>[] summarizers = new Func<List<double>, double>[] { ComputeVolatility, ComputeMin, ComputeMax, ComputeAverage, ComputeRange, ComputeGeoMean };
    public void SaveData(DataManager dataManager, List<BuildName> builds, List<(DataType, string)> chartCriteria = null)
        => SaveData(dataManager, builds, chartCriteria?.Select(s => new List<(DataType, string)> {s}).ToList());

    public void SaveData(DataManager dataManager, List<BuildName> builds, DataType dataType, List<string> chartCriteria = null)
        => SaveData(dataManager, builds, chartCriteria?.Select(s => (dataType, s)).ToList());
    public void SaveData(DataManager dataManager, List<BuildName> builds, DataType dataType, List<List<string>> chartCriteria = null)
        => SaveData(dataManager, builds, chartCriteria?.Select(s => s.Select(s2 => (dataType, s2)).ToList()).ToList());
    public void SaveDataOne(DataManager dataManager, List<BuildName> builds, DataType dataType, List<string> chartCriteria = null)
        => SaveData(dataManager, builds, new List<List<(DataType, string)>>() { chartCriteria?.Select(s => (dataType, s)).ToList() });

    public void SaveData(DataManager dataManager, List<BuildName> builds, List<List<(DataType, string)>> chartCriteria = null)
    {
        // Build Parent -> < Run -> < Benchmark -> Data >>>
        Dictionary<BuildName, Dictionary<string, Dictionary<string, LoadInfo>>> listOfData = new(new BuildNameComparer());

        foreach (var build in builds)
        {
            if (!listOfData.TryGetValue(build, out var b))
            {
                listOfData[build] = b = new();
            }

            foreach (var run in dataManager._runToBenchmarkData)
            {
                if (run.Key.Contains(build.InData))
                {
                    b.Add(run.Key, run.Value);
                }
            }
        }

        // At this point all the data has been categorized.

        // Build Parent -> < DataType -> < Benchmark -> BenchmarkSummaryData >>
        Dictionary<BuildName, Dictionary<string, BenchmarkSummaryData>[]> buildToBenchmarkSummaryData = new(new BuildNameComparer());
            //summarizers.Select(_ => new Dictionary<string, Dictionary<string, BenchmarkSummaryData>>()).ToArray();

        // Get the Summary Data Per Build.
        foreach (var b in listOfData)
        {
            if (!buildToBenchmarkSummaryData.TryGetValue(b.Key, out var data))
            {
                buildToBenchmarkSummaryData[b.Key] = data = summarizers.Select(_ => new Dictionary<string, BenchmarkSummaryData>()).ToArray();
            }

            foreach (var br in dataManager._benchmarkToRunData)
            {
                for (DataType type = DataType.MIN_VALUE; type < DataType.COUNT; ++type)
                {
                    data[(int) type][br.Key] = new();
                }
            }

            Dictionary<string, List<LoadInfo>> benchmarkToData = new();
            foreach (var run in b.Value)
            {
                foreach (var benchmark in run.Value)
                {
                    if (!benchmarkToData.TryGetValue(benchmark.Key, out var d))
                    {
                        benchmarkToData[benchmark.Key] = d = new();
                    }

                    d.Add(benchmark.Value);
                }
            }
        }

        //string DisplayDetailsForABenchmark(BenchmarkSummaryData val) =>
        //    $"{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS},{val.NumberOfHeapCountSwitches},{val.MaxHeapCount}";
        if (chartCriteria == null)
        {
            chartCriteria = new() { new() { (DataType.Volatility, nameof(LoadInfo.MaxPrivateMemoryMB)) } };
        }

        foreach (var (group, criteriaIndex) in chartCriteria.WithIndex())
        {
            Func<KeyValuePair<string, BenchmarkSummaryData>, double> sortingFunctor = null;
            List<Func<BenchmarkSummaryData, double>> selectionFunctors = new();
            List<string> units = new();

            foreach (var ((type, s), index) in group.WithIndex())
            {
            Func<KeyValuePair<string, BenchmarkSummaryData>, double> thisSortingFunctor = null;
            Func<BenchmarkSummaryData, double> selectionFunctor = null;
            string unit = null;
            switch (s)
            {
                //case nameof()
                case nameof(BenchmarkSummaryData.MaxWorkingSetMB):
                    thisSortingFunctor = (data) => data.Value.MaxWorkingSetMB;
                    selectionFunctor = (data) => data.MaxWorkingSetMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.MaxPrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.MaxPrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P99PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P99PrivateMemoryMB;
                    selectionFunctor = (data) => data.P99PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P95PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P95PrivateMemoryMB;
                    selectionFunctor = (data) => data.P95PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P90PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P90PrivateMemoryMB;
                    selectionFunctor = (data) => data.P90PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P75PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P75PrivateMemoryMB;
                    selectionFunctor = (data) => data.P75PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P50PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P50PrivateMemoryMB;
                    selectionFunctor = (data) => data.P50PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.RequestsPerMSec):
                    thisSortingFunctor = (data) => data.Value.RequestsPerMSec;
                    selectionFunctor = (data) => data.RequestsPerMSec;
                    unit = "Req/sec";
                    break;
                case nameof(BenchmarkSummaryData.MeanLatencyMS):
                    thisSortingFunctor = (data) => data.Value.MeanLatencyMS;
                    selectionFunctor = (data) => data.MeanLatencyMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency50thMS):
                    thisSortingFunctor = (data) => data.Value.Latency50thMS;
                    selectionFunctor = (data) => data.Latency50thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency75thMS):
                    thisSortingFunctor = (data) => data.Value.Latency75thMS;
                    selectionFunctor = (data) => data.Latency75thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency90thMS):
                    thisSortingFunctor = (data) => data.Value.Latency90thMS;
                    selectionFunctor = (data) => data.Latency90thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency99thMS):
                    thisSortingFunctor = (data) => data.Value.Latency99thMS;
                    selectionFunctor = (data) => data.Latency99thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.MaxHeapCount):
                    thisSortingFunctor = (data) => data.Value.MaxHeapCount;
                    selectionFunctor = (data) => data.MaxHeapCount;
                    unit = "heap count";
                    break;
                case nameof(BenchmarkSummaryData.NumberOfHeapCountSwitches):
                    thisSortingFunctor = (data) => data.Value.NumberOfHeapCountSwitches;
                    selectionFunctor = (data) => data.NumberOfHeapCountSwitches;
                    unit = "hc switches";
                    break;
                case nameof(BenchmarkSummaryData.NumberOfHeapCountDirectionChanges):
                    thisSortingFunctor = (data) => data.Value.NumberOfHeapCountDirectionChanges;
                    selectionFunctor = (data) => data.NumberOfHeapCountDirectionChanges;
                    unit = "hc dir changes";
                    break;
                case nameof(BenchmarkSummaryData.TotalSuspensionTimeMSec):
                    thisSortingFunctor = (data) => data.Value.TotalSuspensionTimeMSec;
                    selectionFunctor = (data) => data.TotalSuspensionTimeMSec;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.PercentPauseTimeInGC):
                    thisSortingFunctor = (data) => data.Value.PercentPauseTimeInGC;
                    selectionFunctor = (data) => data.PercentPauseTimeInGC;
                    unit = "%";
                    break;
                case nameof(BenchmarkSummaryData.PercentTimeInGC):
                    thisSortingFunctor = (data) => data.Value.PercentTimeInGC;
                    selectionFunctor = (data) => data.PercentTimeInGC;
                    unit = "%";
                    break;
                case nameof(BenchmarkSummaryData.MeanHeapSizeBeforeMB):
                    thisSortingFunctor = (data) => data.Value.MeanHeapSizeBeforeMB;
                    selectionFunctor = (data) => data.MeanHeapSizeBeforeMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.MaxHeapSizeMB):
                    thisSortingFunctor = (data) => data.Value.MaxHeapSizeMB;
                    selectionFunctor = (data) => data.MaxHeapSizeMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.TotalAllocationsMB):
                    thisSortingFunctor = (data) => data.Value.TotalAllocationsMB;
                    selectionFunctor = (data) => data.TotalAllocationsMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.GCScore):
                    thisSortingFunctor = (data) => data.Value.GCScore;
                    selectionFunctor = (data) => data.GCScore;
                    unit = "score";
                    break;

                default:
                    throw new Exception($"unexpected {s}");
                    //thisSortingFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    //selectionFunctor = (data) => data.MaxPrivateMemoryMB;
                    //unit = "MB";
                    //break;
            }
            sortingFunctor = sortingFunctor ?? thisSortingFunctor; // keep first one
            selectionFunctors.Add(selectionFunctor);
            units.Add(unit);
            }

            var uniqueUnits = units.Zip(group.Select(t => t.Item1)).Select(p => p.Item2 == DataType.Volatility ? "Volatility Score" : p.Item1).Distinct();
            if (uniqueUnits.Count() > 2) throw new Exception("More than two units in chart");

            List<PerBuildData> pairedPerBuildData = new();
            List<PerBuildData> sortedPerBuildData = new();

            foreach (BuildName build in buildToBenchmarkSummaryData.Keys)
            {
                for (int groupIndex = 0; groupIndex < group.Count; ++groupIndex)
                {
                    var b = buildToBenchmarkSummaryData[build][(int) group[groupIndex].Item1];
                    var pairedData = b.Zip(buildToBenchmarkSummaryData[buildToBenchmarkSummaryData.Keys.First()][(int) group[0].Item1]).OrderByDescending(pair => sortingFunctor(pair.Second)).Select(pair => pair.First.Value);
                    //pairedPerBuildData.Add(new PerBuildData(group[groupIndex], units[groupIndex], build, selectionFunctors[groupIndex], b.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList()));
                    sortedPerBuildData.Add(new PerBuildData(group[groupIndex], units[groupIndex], build, selectionFunctors[groupIndex], pairedData)); // b.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList() ));
                    //sortedPerBuildData.Add(new PerBuildData(group[groupIndex], units[groupIndex], build, selectionFunctors[groupIndex], b.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList()));
                }
            }


//            // Create CSV.
//            StringBuilder top = new();
//
//            // Iterate over each of the runs.
//            const int singleBuildColumnSize = 10;
//            int numberOfBuilds = buildToBenchmarkSummaryData.Count;
//            string columnHeader = "Benchmark Name,MaxWorkingSetMB,MaxPrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS,# HC Switches";
//
//            // Assumption: the same benchmarks are present for all runs.
//            int totalCountOfBenchmarks = buildToBenchmarkSummaryData.First().Value.Count;
//
//            string first = string.Join(",", namesOfBuilds.Select(build => build + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize))));
//            string second = string.Join(",,", Enumerable.Repeat(columnHeader, numberOfBuilds));
//
//            top.AppendLine(first);
//            top.AppendLine(second);
//
//            for (int benchmarkIdx = 0; benchmarkIdx < totalCountOfBenchmarks; benchmarkIdx++)
//            {
//                top.AppendLine(string.Join(",,", namesOfBuilds.Select(buildName => DisplayDetailsForABenchmark(sortedPerBuildVolatility[buildName][benchmarkIdx]))));
//            }
//
//            File.WriteAllText(Path.Combine(dataManager._basePath, $"Volatility_{group[0]}.csv"), top.ToString());

            // Chart the sorted % Vol Results.

            ColorProvider colorProvider = new();
            //colorProvider.StartColors(builds.Select(build => build.InData));
            List<Scatter> scatters = new();
            //string mode = "markers";
            string mode = "lines+markers";
            string firstUnit = sortedPerBuildData[0].Unit;

            var layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Benchmark Name" },
                yaxis = new Yaxis { title = firstUnit },
                width = 1200,
                title = $"GCMetrcs Sorted by {group[0].Item1} of {group[0].Item2} for {builds[0].ToDisplay} (by test)"
            };

            foreach (var (b, index) in sortedPerBuildData.WithIndex())
            {
                var scatter = new Scatter
                {
                    x = b.Data.Select(s => s.Benchmark),
                    y = b.Data.Select(v => b.Selector(v)),// + 0.1 * index),
                    mode = mode,
                    name = $"{b.BuildName.ToDisplay}: {b.Criteria.Item1.ToString()} of {b.Criteria.Item2}",
                };

                if (b.Unit != firstUnit)
                {
                    layout.yaxis2 = new Yaxis { title = b.Unit, side = "right", overlaying = "y" };
                    scatter.yaxis = "y2";
                }

                colorProvider.SetMarker(scatter, b.BuildName.InData, sortedPerBuildData.Count());
                scatters.Add(scatter);
            }

            Chart.Plot(scatters, layout).Display();

//            scatters.Clear();
//            layout = new Layout.Layout
//            {
//                xaxis = new Xaxis { title = "Benchmark Index" },
//                yaxis = new Yaxis { title = firstUnit },
//                width = 1200,
//                title = $"GCMetrcs Sorted by {group[0].Item1} of {group[0].Item2} for {builds[0].ToDisplay} (by index)"
//            };

//            //colorProvider.StartColors(builds.Select(build => build.InData));
//            foreach (var b in sortedPerBuildData)
//            {
//                var sortedData = b.Data.OrderByDescending(b.Selector);
//                var scatter = new Scatter
//                {
//                    x = Enumerable.Range(0, sortedData.Count()),
//                    y = sortedData.Select(b.Selector),
//                    mode = mode,
//                    name = $"{b.BuildName.ToDisplay}: {b.Criteria.Item1.ToString()} of {b.Criteria.Item2}",
//                    text = sortedData.Select(ss => ss.Benchmark),
//                };

//                if (b.Unit != firstUnit)
//                {
//                    layout.yaxis2 = new Yaxis { title = b.Unit, side = "right", overlaying = "y" };
//                    scatter.yaxis = "y2";
//                }

//                colorProvider.SetMarker(scatter, b.BuildName.InData, sortedPerBuildData.Count());
//                scatters.Add(scatter);
//            }
            
//            Chart.Plot(scatters, layout).Display();
        }
    }
*/

// CompareFull is used to compare different builds.

/*
public class CollectedBenchmarkData
{
    public List<double> Data = new();
    public double Sum => Data.Sum(x => x);
    public double Prod => Data.Aggregate(1.0, (prod, next) => prod * next);
    public double Average => Sum / Data.Count();
    public double GeoMean => Math.Pow(Prod, 1.0 / Data.Count());

    public void Add(double value) => Data.Add(value);
}

public class Blob // rename this...
{
    public CollectedBenchmarkData Baseline = new();
    public List<CollectedBenchmarkData> Diffs = new();
    public double Ratio(int i) => Diffs[i].GeoMean / Baseline.GeoMean;
}

void CheckAdd(string benchmark, CollectedBenchmarkData data, Func<LoadInfo, double> selector, string includeRE, string excludeRE)
{
    if ((includeRE != null) && !Regex.Match(benchmark, includeRE).Success) return;
    if ((excludeRE != null) && Regex.Match(benchmark, excludeRE).Success) return;
    if (!data.TryGetValue(benchmark, out var blob)) data[benchmark] = blob = new Blob();

}

// selector -> to extract the data to CompareFull
// includeRE -> which benchmarks to include (regex), all if null
// excludeRE -> which benchmarks to exclude (regex), none if null
// baseline/diffs -> names of builds to compare
// includeIndiv -> whether to include the individual benchmark comparisons (probably use true)
void CompareFull(DataManager dataManager, Func<LoadInfo, double> selector, string includeRE, string excludeRE, string baseline, List<string> diffs, bool includeIndiv)
{
    HashSet<string> seen = new();
    // benchmark -> Blob
    Dictionary<string, Blob> data = new();
    foreach (var (run, benchmarkData) in dataManager._runToBenchmarkData)
    {
        var build = run.Substring(0, run.LastIndexOf('_'));
        //if (!seen.Contains(build)) { Console.WriteLine(build); seen.Add(build); }
        if (build != baseline && build != diff) continue;
        foreach (var (benchmark, loadInfo) in benchmarkData)
        {
            if ((includeRE != null) && !Regex.Match(benchmark, includeRE).Success) continue;
            if ((excludeRE != null) && Regex.Match(benchmark, excludeRE).Success) continue;
            if (!data.TryGetValue(benchmark, out var blob)) data[benchmark] = blob = new Blob();
            if (build == baseline) blob.Baseline.Add(selector(loadInfo));
            else 
            blob.GetData(build == baseline).Add(selector(loadInfo));
        }
    }

    List<double> ratios = new();
    Console.WriteLine($"Baseline: {baseline}");
    foreach (var (d, i) in diffs.WithIndex())
    {
        Console.WriteLine($"Diff{i}: {d}");
    }
    {
        Console.WriteLine($"{"Benchmark",35} | {"D/B",5} | {"Base",8} | {"Diff",8}");
        Console.WriteLine($"{new string('-', 35)}-+-{new string('-', 5)}-+-{new string('-', 8)}-+-{new string('-', 8)}");
        foreach (var (benchmark, value) in data.OrderByDescending(kvp => kvp.Value.Ratio))
        {
            if (includeIndiv)
            {
                Console.WriteLine($"{benchmark,35} | {value.Ratio,5:N3} | {value.Baseline.GeoMean,8:N2} | {value.Diff.GeoMean,8:N2}");
            }
            ratios.Add(value.Ratio);
        }
    }

    Console.WriteLine($"{new string('-', 35)}-+-{new string('-', 5)}-+-{new string('-', 8)}-+-{new string('-', 8)}");
    double baseGeoMean = ComputeGeoMean(data.Select(kvp => kvp.Value.Baseline.GeoMean));
    double diffGeoMean = ComputeGeoMean(data.Select(kvp => kvp.Value.Diff.GeoMean));
    Console.WriteLine($"{"GeoMean",35} | {diffGeoMean / baseGeoMean,5:N3} | {baseGeoMean,8:N2} | {diffGeoMean,8:N2}");
    Console.WriteLine($"{"ArithMean",35} | {ComputeAverage(ratios),5:N3} | {"",8} | {"",8}");
    Console.WriteLine();
}
*/

// Display individual benchmark runs

// extract -> to extract the data to CompareFull
// benchmark -> benchmarks(s) to include
// exactMatch -> impact matching of benchmark - odd behavior.. see code

/*
void ProcessDataMean(DataManager dataManager, Func<LoadInfo, double> extract, string benchmarkName, bool exactMatch = false)
{
    Console.WriteLine("Benchmark {0}", benchmarkName);
    var names = dataManager.Data.Keys;
    // build -> (sum of GCMetrc, count)
    Dictionary<string, (double sum, int count)> GCMetrcByBuild = new(2);
    foreach (var name in names)
    {
        bool matched = (exactMatch ? name.EndsWith(benchmarkName) : name.Contains(benchmarkName));
        if (matched)
        {
            string[] fields = name.Split(new Char[] { '_' }, StringSplitOptions.RemoveEmptyEntries);
            string buildname = fields[0];
            LoadInfo info = dataManager.Data[name];
            //Console.WriteLine("build name is {0}", buildname);
            if (GCMetrcByBuild.TryGetValue(buildname, out (double sum, int count) p))
            {
                metricByBuild[buildname] = (p.sum + extract(info), p.count + 1);
            }
            else
            {
                metricByBuild.Add(buildname, (extract(info), 1));
            }

            //Console.WriteLine("metric is now {0:0.00}", metricByBuild[buildname]);

            Console.WriteLine("{0,60}: mean latency {1:0.00} ms, time in GC {2:0.00}%, heap switches {3}, max HC {4,2}, maxHeapMB {5:0.0}",
                name, info.MeanLatencyMS, info.PercentPauseTimeInGC,
                info.NumberOfHeapCountSwitches, info.MaxHeapCount, info.MaxHeapSizeMB);
        }
    }

    foreach (var (build, (sum, count)) in metricByBuild)
    {
        Console.WriteLine("build {0, 10}: {1:0.00}", build, sum / count);
    }
}

void ProcessDataMeanLatency(DataManager dataManager, string benchmarkName, bool exactMatch = false)
    => ProcessDataMean(dataManager, loadInfo => loadInfo.MeanLatencyMS, benchmarkName, exactMatch);
*/

// Shows benchmark runs that increment the metric two times in a row

// builds -> the build name (with trailing _, like "fix_")
// extract -> data to examine
// benchmarkFilterRE -> benchmarks to include
// listAll -> show all data points instead of just the changes - usually too much
/*
public void DisplayDoubleIncrement(DataManager dataManager, IEnumerable<string> builds, Func<TraceGC, double> extract, string benchmarkFilterRE, bool listAll = false)
{
    foreach ((string benchmark, var allRuns) in dataManager._benchmarkToRunData)
    {
        bool firstForBenchmark = true;
        if (!Regex.IsMatch(benchmark, benchmarkFilterRE)) continue;
        foreach ((string run, var results) in allRuns)
        {
            if (!builds.Any(b => run.StartsWith(b))) continue;
            if (results.Data == null)
            {
                Console.WriteLine($"No data for {benchmark} {run}");
                continue;
            }
            var doubleIncr =
                results.Data.GCs
                    .Where(gc => gc.GlobalHeapHistory != null)
                    .Select(extract)
                    .SlidingRange(10)
                    .SkipWhile(window => ((window[0] + 1) != window[3]) || ((window[3] + 1) != window[6])|| ((window[6] + 1) != window[9]));
            bool anyDouble = doubleIncr.Any(x => true);
            if (!anyDouble) continue;

            if (firstForBenchmark)
            {
                Console.WriteLine($"{benchmark}:");
                firstForBenchmark = false;
            }
            Console.Write($" {run,12}:");
            if (listAll)
            {
                foreach (int num in
                    results.Data.GCs
                        .Where(gc => gc.GlobalHeapHistory != null)
                        .Select(extract)
                        .SlidingWindow(-1)
                        .Where(window => window.PrevItem != window.CurrentItem)
                        .Select(window => window.CurrentItem))
                {
                    Console.Write($" {num}");
                }
            }
            Console.WriteLine();
        }
    }
}


// Shows benchmark runs that decrease a metric (ever)

// builds -> the build name (with trailing _, like "fix_")
// extract -> data to examine
// amount -> threshold of decrease to include
// benchmarkFilterRE -> benchmarks to include
// listAll -> show all data points instead of just the changes - usually too much

public void DisplayChangesDown(DataManager dataManager, IEnumerable<string> builds, Func<TraceGC, double> extract, double amount, string benchmarkFilterRE, bool listAll = false)
{
    foreach ((string benchmark, var allRuns) in dataManager._benchmarkToRunData)
    {
        bool firstForBenchmark = true;
        if (!Regex.IsMatch(benchmark, benchmarkFilterRE)) continue;
        foreach ((string run, var results) in allRuns)
        {
            if (!builds.Any(b => run.StartsWith(b))) continue;
            if (results.Data == null)
            {
                Console.WriteLine($"No data for {benchmark} {run}");
                continue;
            }
            var dec =
                results.Data.GCs
                    .Where(gc => gc.GlobalHeapHistory != null)
                    .Select(extract)
                    .SlidingWindow(-1)
                    .SkipWhile(window => window.PrevItem <= window.CurrentItem + amount);
            bool anyDecrease = dec.Any(x => true);
            if (!anyDecrease) continue;

            var incAfterDec = dec.SkipWhile(window => window.PrevItem >= window.CurrentItem - amount);
            var anyIncAfterDec = incAfterDec.Any(x => true);
            if (!anyIncAfterDec) continue;

            if (firstForBenchmark)
            {
                Console.WriteLine($"{benchmark}:");
                firstForBenchmark = false;
            }
            Console.Write($" {run,12}:");
            if (listAll)
            {
                foreach (int num in
                    results.Data.GCs
                        .Where(gc => gc.GlobalHeapHistory != null)
                        .Select(extract)
                        .SlidingWindow(-1)
                        .Where(window => window.PrevItem != window.CurrentItem)
                        .Select(window => window.CurrentItem))
                {
                    Console.Write($" {num}");
                }
            }
            Console.WriteLine();
        }
    }
}

public void DisplayHeapChangesDown(DataManager dataManager, IEnumerable<string> builds, string benchmarkFilterRE)
    => DisplayChangesDown(dataManager, builds, gc => gc.GlobalHeapHistory.NumHeaps, 0, benchmarkFilterRE, listAll: true);

// This is used to reduce a list of charts into a small enough number for the notebook to fully display.
// I have a habit of calling GetPage(0) for small lists, but this isn't needed - just display the whole list.
// (When displaying all benchmarks, having that and changing the 0 to 1, 2, etc., can be useful)
static IEnumerable<PlotlyChart> GetPage(this IEnumerable<PlotlyChart> groups, int page, int numPerPage = 18)
    => groups.Skip(numPerPage * page).Take(numPerPage);
*/


In [ ]:
// Old examples - use as resources then delete

/*

SummarizeResultsByBench(low4DM, ML("v2-fixrearranged-all", "v2-fixrearranged-all-svr"));
SummarizeResultsByBench(low4DM, ML("v2-rc3", "v2-fixrearranged"));

SummarizeResults(diffDataManager, latestPath + @"\summarize.txt");

// The specific values are busted here, but more paths can be added to an existing DataManager.
// Note: Adding/overwriting more benchmarks to an existing loaded directory is untested/etc.
// This is intended for adding a new run when you already have a baseline or previous run
// loaded and don't want to wait to read it again.

dataManager.AddData(new[] { slopePath, evalDecrPath }, scoutList.ToList());

// Again, the values are busted, but you can speed up loading if you only want to look at
// certain benchmarks.

var x = new DataManager(new[] { evalDecrPath }, filter: debugList);

var low4BaseRun = ML(new BuildName("v2-fixrearranged-mult-max_", "base"));
var low4Run = ML(new BuildName("v2-fixrearranged-mult-max-h4_", "max4"));
var svrRun = ML(new BuildName("v2-fixrearranged-mult-max-svr_", "svr"));
var svr4Run = ML(new BuildName("v2-fixrearranged-mult-max-svr4_", "svr4"));
var mult8Run = ML(new BuildName("v2-fixrearranged-mult-max-mult8_", "mult8"));
var mult32Run = ML(new BuildName("v2-fixrearranged-mult-max-mult32_", "mult32"));
var mult8max10Run = ML(new BuildName("v2-fixrearranged-mult-max-mult8x10_", "m8x10"));
var max10Run = ML(new BuildName("v2-fixrearranged-mult-max-x10_", "x10"));
var low4CompRuns = Concat(low4BaseRun, low4Run, svrRun, svr4Run, mult8Run, mult32Run, mult8max10Run, max10Run);

string compareBase = "v2-fixrearranged-mult-max-svr4";
string compareDiff = "v2-fixrearranged-mult-max-mult8";

//var extract = (LoadInfo loadInfo) => loadInfo.RequestsPerMSec;
//var extract = (LoadInfo loadInfo) => loadInfo.Latency50thMS;
var extract = (LoadInfo loadInfo) => loadInfo.MeanLatencyMS;
//var extract = (LoadInfo loadInfo) => loadInfo.MaxPrivateMemoryMB;
//var extract = (LoadInfo loadInfo) => loadInfo.P50PrivateMemoryMB;
//var extract = (LoadInfo loadInfo) => loadInfo.PercentPauseTimeInGC;

string includeRE = null; // scoutREListShort2;
string excludeRE = null; // "ConnectionClose";
CompareFull(low4DM, extract, includeRE, excludeRE, compareBase, compareDiff, true);
//CompareFull(rc3DataManager, (LoadInfo loadInfo) => Math.Max(5, loadInfo.PercentPauseTimeInGC), scoutREListShort, null, compareBase, compareDiff, true);

foreach (string benchmark in new[] { "ConnectionClose", "SingleQueryPlatform" })
{
    ProcessDataMeanLatency(rc2DataManager, benchmark, true);
}

// Not tested for a while.

DisplayDoubleIncrement(rc2DataManager, ML("fix_"), gc => gc.GlobalHeapHistory.NumHeaps, "", true)

// Not tested for a while

DisplayChangesDown(rc2DataManager, ML("fix_"), gc => gc.HeapSizeAfterMB, 3, "")

// Leftover code - manually displays heap changes

foreach (string build in ML("v2-fixrearranged-all_"))
{
    for (int i = 0; i < 3; ++i)
    {
        Console.Write($"{build}{i}:");
        foreach (int num in
            low4DM.Data[$"{build}{i} | Stage1"].Data.GCs
                .Where(gc => gc.GlobalHeapHistory != null)
                .Select(gc => gc.GlobalHeapHistory.NumHeaps)
                .SlidingWindow(-1)
                .Where(window => window.PrevItem != window.CurrentItem)
                .Select(window => window.CurrentItem))
                {
                    Console.Write($" {num}");
                }
        Console.WriteLine();
    }
}

*/

In [ ]:
// Old charting examples
/*
SaveData(low4DM, low4CompRuns, ML((DataType.Average, nameof(LoadInfo.P90PrivateMemoryMB)), (DataType.Max, nameof(LoadInfo.MaxPrivateMemoryMB)), (DataType.Max, nameof(LoadInfo.MaxHeapCount))));
SaveData(low4DM, low4CompRuns, DataType.Average, ML(nameof(LoadInfo.RequestsPerMSec)));
SaveData(dataManager, allRuns, DataType.Average, ML(nameof(LoadInfo.RequestsPerMSec), nameof(LoadInfo.Latency50thMS)));
SaveData(rc3DataManager, rc3RearrangedRun,
    ML(ML((DataType.Max, nameof(LoadInfo.MaxPrivateMemoryMB)), (DataType.Min, nameof(LoadInfo.MaxPrivateMemoryMB))),
        ML((DataType.Max, nameof(LoadInfo.P99PrivateMemoryMB)), (DataType.Min, nameof(LoadInfo.P99PrivateMemoryMB))),
        ML((DataType.Max, nameof(LoadInfo.P50PrivateMemoryMB)), (DataType.Min, nameof(LoadInfo.P50PrivateMemoryMB)))));
SaveData(rc3DataManager, rc3Runs, DataType.Average, priMemList);
SaveData(diffDataManager, vsBaseRuns, priMemList.Select(m => ML((DataType.Min, m), (DataType.Max, m))));
SaveDataOne(v2DataManager, v2Runs, DataType.Average, priMemList);
SaveDataOne(diffDataManager, allRuns, DataType.Volatility, volList);

// Using the DataManager - I haven't been using this section.

// The following cells demonstrates how to make use of the ``DataManager``. 

// The name of the run from the yaml file for which the ASP.NET run is created for.
string runName = "base_0";

Dictionary<string, LoadInfo> run = dataManager.GetAllBenchmarksForRun(runName);
dataManager.Data.Display();
List<KeyValuePair<string, LoadInfo>> runsWithGCData = dataManager.GetAllBenchmarksForRun(runName).Where(gc => gc.Value.Data != null);

string benchmarkName = "Stage2";
LoadInfo benchmarkData = dataManager.GetBenchmarkData(benchmark: benchmarkName, run: runName);
benchmarkData.Id

Dictionary<string, LoadInfo> allRunsForBenchmark = dataManager.GetAllRunsForBenchmark(benchmark: benchmarkName);
allRunsForBenchmark.Keys

dataManager.SaveBenchmarkData()

// ## Build to Build Comparison and Volatility Analysis

// I haven't been using this section, but it is an obvious one to start using again.

var run1_vs_run2 = diffDataManager.GetBenchmarkToComparison("tp3-m_0", "tp3-m_1");

static bool IsNotInvalidDouble(double val) => 
    !double.IsNaN(val) && 
    !double.IsInfinity(val) && 
    !double.IsPositiveInfinity(val) && 
    !double.IsNegativeInfinity(val);

public class SummaryTable
{
    public SummaryTable(Dictionary<string, Dictionary<string, LoadInfo>> comparisons)
    {
        Comparisons = comparisons;
    }

    private string GenerateSummaryForComparison(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        double averageWorkingSet = comparison.Where(a => IsNotInvalidDouble(a.Value.MaxWorkingSetMB)).Average(a => a.Value.MaxWorkingSetMB);
        double privateMemory = comparison.Where(a => IsNotInvalidDouble(a.Value.MaxPrivateMemoryMB)).Average(a => a.Value.MaxPrivateMemoryMB);
        double throughput = comparison.Where(a => IsNotInvalidDouble(a.Value.RequestsPerMSec)).Average(a => a.Value.RequestsPerMSec);
        double meanLatency = comparison.Where(a => IsNotInvalidDouble(a.Value.MeanLatencyMS)).Average(a => a.Value.MeanLatencyMS);

        double p50Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency50thMS)).Average(a => a.Value.Latency50thMS);
        double p75Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency75thMS)).Average(a => a.Value.Latency75thMS);
        double p90Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency90thMS)).Average(a => a.Value.Latency90thMS);
        double p99Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency99thMS)).Average(a => a.Value.Latency99thMS);

        return $"{comparisonKey},{averageWorkingSet},{privateMemory},{throughput},{meanLatency},{p50Latency},{p75Latency},{p90Latency},{p99Latency}";
    }

    public string GenerateSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Average Max Working Set (MB) %, Average Max Private Memory (MB) %, Average Request/MSec %, Average Mean Latency (MSec), Average P50 Latency (MSec) %, Average P75 Latency (MSec) %, Average P90 Latency (MSec) %, Average P99 Latency (MSec) %");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateSummaryForComparison(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    private int GetCountOfRegressions(List<double> selected, double thresholdPercentage, bool lessIsBetter = true)
    {
        // If throughput, less is worse => threshold <= -5%.
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && ( (lessIsBetter) ? (d >= thresholdPercentage) : (d <= -thresholdPercentage)));
        return comparison.Count;
    }

    private int GetCountOfAbsRegressions(List<double> selected, double thresholdPercentage)
    {
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && Math.Abs(d) >= thresholdPercentage);
        return comparison.Count;
    }

    // # of benchmarks with throughput regressed by >= 5% and 10%
    private string GenerateRegressionSummary(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        List<double> workingSet  = comparison.Select(c => c.Value.MaxWorkingSetMB);
        int workingSetCountGT_5  = GetCountOfRegressions(workingSet, 5);
        int workingSetCountGT_10 = GetCountOfRegressions(workingSet, 10);

        List<double> privateMemory  = comparison.Select(c => c.Value.MaxPrivateMemoryMB);
        int privateMemoryCountGT_5  = GetCountOfRegressions(privateMemory, 5);
        int privateMemoryCountGT_10 = GetCountOfRegressions(privateMemory, 10);

        List<double> throughput  = comparison.Select(a => a.Value.RequestsPerMSec);
        int throughputCountGT_5  = GetCountOfRegressions(throughput, 5, false);
        int throughputCountGT_10 = GetCountOfRegressions(throughput, 10, false);

        List<double> meanLatency  = comparison.Select(a => a.Value.MeanLatencyMS);
        int meanLatencyCountGT_5  = GetCountOfRegressions(meanLatency, 5);
        int meanLatencyCountGT_10 = GetCountOfRegressions(meanLatency, 10);

        List<double> p50Latency  = comparison.Select(a => a.Value.Latency50thMS);
        int p50LatencyCountGT_5  = GetCountOfRegressions(p50Latency, 5);
        int p50LatencyCountGT_10 = GetCountOfRegressions(p50Latency, 10);

        List<double> p75Latency  = comparison.Select(a => a.Value.Latency75thMS);
        int p75LatencyCountGT_5  = GetCountOfRegressions(p75Latency, 5);
        int p75LatencyCountGT_10 = GetCountOfRegressions(p75Latency, 10);

        List<double> p90Latency  = comparison.Select(a => a.Value.Latency90thMS);
        int p90LatencyCountGT_5  = GetCountOfRegressions(p90Latency, 5);
        int p90LatencyCountGT_10 = GetCountOfRegressions(p90Latency, 10);
        
        List<double> p99Latency  = comparison.Select(a => a.Value.Latency99thMS);
        int p99LatencyCountGT_5  = GetCountOfRegressions(p99Latency, 5);
        int p99LatencyCountGT_10 = GetCountOfRegressions(p99Latency, 10);

        return $"{comparisonKey},{workingSetCountGT_5},{workingSetCountGT_10},{privateMemoryCountGT_5},{privateMemoryCountGT_10},{throughputCountGT_5},{throughputCountGT_10},{meanLatencyCountGT_5},{meanLatencyCountGT_10},{p50LatencyCountGT_5},{p50LatencyCountGT_10},{p75LatencyCountGT_5},{p75LatencyCountGT_10},{p90LatencyCountGT_5},{p90LatencyCountGT_10},{p99LatencyCountGT_5},{p99LatencyCountGT_10}";
    }

    public string GenerateRegressionSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Reg. Count - Working Set (MB),Large Reg. Count - Working Set (MB),Reg. Count - Max Private Memory (MB),Large Reg. Count - Max Private Memory (MB),Reg. Count - Throughput, Large Reg. Count - Throughput,Reg. Count - Mean Latency,Large Reg. Count - Mean Latency,Reg. Count - P50 Latency, Large Reg. Count - P50 Latency, Reg. Count - P75 Latency, Large Reg. Count - P75 Latency,Reg. Count - P90 Latency, Large Reg. Count - P90 Latency,Reg. Count - P99 Latency, Large Reg. Count - P99 Latency");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateRegressionSummary(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    public Dictionary<string, string> GenerateRegressionAnalysisForComparison(string comparisonKey)
    {
        StringBuilder sb = new();
        Dictionary<string, string> csvData = new();
        Dictionary<string, LoadInfo> comparison = Comparisons[comparisonKey];

        string header = "Benchmark,MaxWorkingSetMB,MaxPrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS";

        // Generate Memory Regressions.
        StringBuilder memRegressions = new();
        memRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MaxWorkingSetMB >= 10 || c.Value.MaxPrivateMemoryMB >= 10 ))
        {
            memRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["memory"] = memRegressions.ToString();

        // Generate Throughput Regressions.
        StringBuilder throughputRegressions = new();
        throughputRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.RequestsPerMSec <= -10))
        {
            throughputRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["throughput"] = throughputRegressions.ToString();

        // Generate Latency Regressions.
        StringBuilder latencyRegressions = new();
        latencyRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MeanLatencyMS >= 10 || 
                                                        c.Value.Latency50thMS >= 10 || 
                                                        c.Value.Latency75thMS >= 10 || 
                                                        c.Value.Latency90thMS >= 10 || 
                                                        c.Value.Latency99thMS >= 10 ))
        {
            latencyRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["latency"] = latencyRegressions.ToString();

        // All.
        StringBuilder all = new();
        all.AppendLine(header);
        foreach (var benchmark in comparison)
        {
            all.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["all"] = all.ToString();

        return csvData;
    }

    public void SaveComparisons(string basePath)
    {
        // Add Summary for Comparisons.
        string summaryOfComparisons = GenerateSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "SummaryOfComparisons.csv"), summaryOfComparisons);

        // Add Regression Summary for Comparisons.
        string regressionSummary = GenerateRegressionSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "RegressionSummary.csv"), regressionSummary);

        // Add Large Regression Analysis for Comparison.
        string perComparisonDataPath = Path.Combine(basePath, "PerComparisonData");
        if (!Directory.Exists(perComparisonDataPath))
        {
            Directory.CreateDirectory(perComparisonDataPath);
        }

        foreach (var comparison in Comparisons)
        {
            string comparisonPath = Path.Combine(perComparisonDataPath, comparison.Key);
            Directory.CreateDirectory(comparisonPath);

            Dictionary<string, string> regressionComparisons = GenerateRegressionAnalysisForComparison(comparison.Key);

            // Memory
            File.WriteAllText(Path.Combine(comparisonPath, "MemoryRegressions.csv"), regressionComparisons["memory"]);

            // Throughput
            File.WriteAllText(Path.Combine(comparisonPath, "ThroughputRegressions.csv"), regressionComparisons["throughput"]);

            // Latency
            File.WriteAllText(Path.Combine(comparisonPath, "LatencyRegressions.csv"), regressionComparisons["latency"]);

            // All
            File.WriteAllText(Path.Combine(comparisonPath, "All.csv"), regressionComparisons["all"]);
        }
    }

    public Dictionary<string, Dictionary<string, LoadInfo>> Comparisons { get; }
}


//var datas3_vs_datas_4 = baseDataManager.GetBenchmarkToComparison("base_0", "base_1");

Dictionary<string, Dictionary<string, LoadInfo>> comparisons = new()
{
    { nameof(run1_vs_run2), run1_vs_run2 },
};

SummaryTable summaryTable = new(comparisons);
summaryTable.SaveComparisons(diffPath);


// I don't use this anymore (or GCCharting at all)

void ChartProperty(LoadInfo baseline, LoadInfo comparand, string nameOfProperty)
{
    GCProcessData baselineGC = baseline.Data;
    GCProcessData comparandGC = comparand.Data;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for {baseline.Id}" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for {comparand.Id}" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison Between {baseline.Run} and {comparand.Run}", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

void ChartProperty(LoadInfo comparison, string nameOfProperty)
{
    GCProcessData baselineGC = comparison.Data;
    GCProcessData comparandGC = comparison.Data2;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for Baseline" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for Comparand" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

void ChartProperty(IEnumerable<LoadInfo> info, string nameOfProperty)
{
    List<(string scatterName, List<TraceGC> gcs)> gcData =
        info.Select(li => (scatterName: $"{nameOfProperty}", gcs: li.Data.GCs)).ToList();
    GCCharting.ChartGCData(gcData: gcData, title: "${nameOfProperty} Comparison", isXAxisRelative: false, fieldName: nameOfProperty).Display();
}


var run1_Benchmark = diffDataManager.GetBenchmarkData(benchmark: "CachingPlatform", "tp3-m_0");
var run2_Benchmark = diffDataManager.GetBenchmarkData(benchmark: "CachingPlatform", "tp3-m_1");

// Chart the PauseDurationMSec for the run1 vs. run2.
ChartProperty(baseline: run1_Benchmark, comparand: run2_Benchmark, nameof(TraceGC.HeapCount))


// Leftover code that bucketed ranges of values for metrics and displayed them in columns
// - probably out-of-date (and very hardwired to the data I was looking at) - probably ignore this

int[] ranges = {1,11,12,13};

for (int i = 0; i < 4; ++i)
{
    string trace = "fixed-newlinear-nosmooth_" + i;
    Console.Write($"{trace}: ");
    //var cpData = diffDataManager.GetBenchmarkData("MultipleQueriesPlatform", trace);
    var cpData = noDataManager.GetBenchmarkData("Fortunes", trace);

    int prevNumHeaps = -1;
    int count = 0;
    int nextRangeIndex = 0;
    foreach (int numHeaps in cpData.Data.GCs.Select(gc => gc.GlobalHeapHistory?.NumHeaps).Where(x => x.HasValue).Append(-1))
    {
        if (numHeaps == prevNumHeaps)
        {
            count++;
            continue;
        }

        if (count != 0)
        {
            int skip = ranges.Skip(nextRangeIndex).TakeWhile(r => prevNumHeaps > r).Count();
            Console.Write(new string(' ', skip * 13));
            nextRangeIndex += skip + 1;
            Console.Write($"{count,5} @ {prevNumHeaps,2} {(numHeaps == -1 ? ' ' : (numHeaps > prevNumHeaps ? '^' : 'v'))} ");
        }
        prevNumHeaps = numHeaps;
        count = 1;
    }
    Console.WriteLine();
}
*/

## Debugging

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id

In [ ]:
#!about